In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("sparksql-aggregations").getOrCreate()

df = spark.read.parquet("gs://bucket_name/bikeshare_trips")

In [ ]:
df.show()

In [ ]:
import re
def clean_string(string):
    string = string.strip()
    string = re.sub('[^A-Za-z0-9\s]+', '', string)
    return string

In [ ]:
udf_clean_string = udf(lambda x: clean_string(x))

In [ ]:
df.select([udf_clean_string(col('subscriber_type')),"subscriber_type"]).show(10,False)

In [ ]:
@udf(returnType=StringType())
def clean_string_annotation_udf(string):
    string = string.strip()
    string = re.sub('[^A-Za-z0-9\s]+', '', string)
    return string

In [ ]:
df.select([clean_string_annotation_udf(col('subscriber_type')),"subscriber_type"]).show(10,False)

In [ ]:
spark.udf.register("sparksql_udf_clean_string",clean_string)

In [ ]:
df.createOrReplaceTempView("bikeshare")

In [ ]:
spark.sql("select subscriber_type,sparksql_udf_clean_string(subscriber_type) from bikeshare limit 10").show(10,False)